## -----------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------- TESTING ------------------------------------------------------
## ------------------------------------------------------------------------------------------------------------------------------

In [11]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from os import path
import torch

device = 'cuda'


In [21]:
model2 = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
save_path = path.join("saved_model")
model2.load_state_dict(torch.load(save_path))
model2.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [13]:
#!pip install tensorflow_datasets

In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [15]:
# Construct a tf.data.Dataset
ds = tfds.load('goemotions', split='test', shuffle_files=False)

In [16]:
#save the dataframe into csv file
save_path = path.join("go_emotions_test.csv")
df = tfds.as_dataframe(ds)
#read the csv file as normal, then you have the df you need
# import pandas as pd
# df = pd.read_csv(save_path, header = 0, sep=",")
# df

In [17]:
bytelist = df.comment_text.values.tolist()

In [18]:
stringlist=[x.decode('utf-8') for x in bytelist]

In [19]:
from tqdm import tqdm
from transformers import BertTokenizer
import transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
MAX_LEN = 64
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
sentences = stringlist
#labels = df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in tqdm(sentences):
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in tqdm(input_ids):
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
#prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


Number of test sentences: 5,427



100%|███████████████████████████████████████████████████████████████████████████| 5427/5427 [00:00<00:00, 40027.59it/s]


In [22]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model2.eval()
# Tracking variables 
predictions=[]

# Predict 
for batch in tqdm(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model2(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  #label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  #true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 5,427 test sentences...


100%|████████████████████████████████████████████████████████████████████████████████| 170/170 [00:18<00:00,  9.01it/s]

    DONE.


In [24]:
import numpy as np 
from scipy.special import softmax
answers = []
for i in tqdm(range(len(predictions))):
    for j in range(len(predictions[i])):
        answers.append(softmax(predictions[i][j]))
        
final_answers = []
for i in range(len(answers)):
    final_answers.append(np.argmax(answers[i]))

100%|███████████████████████████████████████████████████████████████████████████████| 170/170 [00:00<00:00, 618.54it/s]


In [ ]:
# Confusion Matrix 
from sklearn.metrics import confusion_matrix
y_true = df.label.values

confusion_matrix(y_true, final_answers)


In [32]:
emotions = []
for col in df.columns:
    emotions.append(col)
emotions.remove("comment_text")
emotions

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'neutral',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise']

In [28]:
df.head()

,admiration,amusement,anger,annoyance,approval,caring,comment_text,confusion,curiosity,desire,...,love,nervousness,neutral,optimism,pride,realization,relief,remorse,sadness,surprise
0,False,False,False,False,True,False,"b""You're right, thanks for pointing that out, ...",False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,b'Molon labe!!!',False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,b'So this is what edging feels like',False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,b'Just speak with a Sheffield accent lmao',False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,b'My husband and I have been circling the drai...,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [ ]:
sample['label'] = final_answers

In [ ]:
sample = pd.read_csv('test.csv', delimiter="\t")

In [ ]:
sample.to_csv('samplesubmission.csv',index = False)
sample.head()